In [1]:
!pip install -q langchain-openai langchain-community langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 5.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests=

In [5]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=08fdda8b358f7fd687c4d14171d6b93ae9a01c3ed450d16e713ddef57c928337
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


In [2]:
import os
from typing import TypedDict, Literal

from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper
from langchain.chains import LLMMathChain
from langgraph.graph import StateGraph, END

In [18]:
# --- Environment and LLM Setup ---
os.environ["SERPAPI_API_KEY"] = "28c7641827dc44957197ef674a2945932ec69475db9f5b7af4ab08fe9d6fbff5"

llm = ChatOpenAI(
    api_key="KEY",
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-chat-v3.1:free",
    temperature=0.7,
    streaming=True
)

# --- Tool Definitions ---
search_tool = SerpAPIWrapper()
math_tool = LLMMathChain.from_llm(llm=llm, verbose=False)

In [19]:
# --- State Definition ---
# Updated state to include a route for travel and a result.
class AgentState(TypedDict):
    topic: str
    explanation: str
    summary: str
    calculator_result: str
    travel_result: str
    route: Literal["research", "calculate", "travel"] # The planner's decision

In [20]:
def planner_agent(state: AgentState) -> dict:
    """
    This agent decides whether the user's topic is a math, research, or travel question.
    """
    print("---PLANNER---")
    topic = state["topic"]

    prompt = ChatPromptTemplate.from_template(
        """You are a planner agent. Your job is to determine the best path to take based on the user's query.

        Query: {topic}

        Based on the query, is this a math problem, a travel-related query (like finding hotels or flights), or a general research question?
        Respond with 'calculate', 'travel', or 'research'.
        """
    )

    chain = prompt | llm
    result = chain.invoke({"topic": topic})
    decision = result.content.strip().lower()

    print(f"Planner's Decision: {decision}")
    if "calculate" in decision:
        return {"route": "calculate"}
    elif "travel" in decision:
        return {"route": "travel"}
    else:
        return {"route": "research"}

In [21]:
def researcher_agent(state: AgentState) -> dict:
    """
    This agent uses a web search tool to find information on a topic and then explains it.
    """
    print("---RESEARCHER (with SerpApi Web Search)---")
    topic = state["topic"]

    search_results = search_tool.run(topic)
    print(f"Search Results:\n{search_results}")

    prompt = ChatPromptTemplate.from_template(
        """You are a helpful research assistant. Based on the following search results,
        provide a brief, easy-to-understand explanation of the topic: {topic}.

        Search Results:
        {search_results}
        """
    )

    chain = prompt | llm
    result = chain.invoke({"topic": topic, "search_results": search_results})

    print(f"Researcher's Explanation:\n{result.content}")
    return {"explanation": result.content}

In [22]:
def calculator_agent(state: AgentState) -> dict:
    """
    This agent uses an LLM Math tool to solve a math problem.
    """
    print("---CALCULATOR---")
    topic = state["topic"]

    result = math_tool.invoke({"question": topic})
    answer = result.get('answer', 'Could not calculate answer.')

    print(f"Calculator's Result: {answer}")
    return {"calculator_result": answer}

In [23]:
def travel_planner_agent(state: AgentState) -> dict:
    """
    This agent searches for travel-related information like hotels or flights.
    """
    print("---TRAVEL PLANNER---")
    topic = state["topic"]

    # For this example, we'll just use the general search tool.
    # A real-world application would use a dedicated travel API.
    search_results = search_tool.run(f"Find information about: {topic}")

    print(f"Travel Search Results:\n{search_results}")

    prompt = ChatPromptTemplate.from_template(
        """You are a helpful travel assistant. Summarize the following information for the user's travel query about '{topic}'.

        Search Results:
        {search_results}
        """
    )

    chain = prompt | llm
    result = chain.invoke({"topic": topic, "search_results": search_results})

    print(f"Travel Planner's Response:\n{result.content}")
    return {"travel_result": result.content}

In [24]:
def summarizer_agent(state: AgentState) -> dict:
    """
    This agent takes an explanation and summarizes it in one sentence.
    """
    print("---SUMMARIZER---")
    explanation = state["explanation"]

    prompt = ChatPromptTemplate.from_template(
        "You are a summarization expert. Condense the following text into a single, concise sentence:\n\n{explanation}"
    )

    chain = prompt | llm
    result = chain.invoke({"explanation": explanation})

    print(f"Summarizer's Output:\n{result.content}")
    return {"summary": result.content}

In [25]:
# --- Conditional Routing Function ---
def should_route(state: AgentState) -> Literal["research", "calculate", "travel"]:
    """This function is the decision point for our conditional edge."""
    return state["route"]

In [26]:
# --- Graph Definition ---

workflow = StateGraph(AgentState)

# Add all the agent functions as nodes
workflow.add_node("planner", planner_agent)
workflow.add_node("researcher", researcher_agent)
workflow.add_node("calculator", calculator_agent)
workflow.add_node("travel_planner", travel_planner_agent)
workflow.add_node("summarizer", summarizer_agent)

# Set the entry point to the planner
workflow.set_entry_point("planner")

# Add the conditional edge for the planner
workflow.add_conditional_edges(
    "planner",
    should_route,
    {
        "research": "researcher",
        "calculate": "calculator",
        "travel": "travel_planner",
    },
)

# Define the standard edges
workflow.add_edge("researcher", "summarizer")
workflow.add_edge("summarizer", END)
workflow.add_edge("calculator", END)
workflow.add_edge("travel_planner", END) # Travel result is final

# Compile the graph
app = workflow.compile()

In [27]:
# --- Main Execution Block ---
if __name__ == "__main__":
    topic = input("Please enter a topic, math problem, or travel query: ")

    inputs = {"topic": topic}
    final_state = app.invoke(inputs)

    print("\n--- FINAL RESULT ---")
    # The final result depends on which path was taken
    if final_state.get('summary'):
        print(final_state['summary'])
    elif final_state.get('calculator_result'):
        print(final_state['calculator_result'])
    elif final_state.get('travel_result'):
        print(final_state['travel_result'])
    else:
        print("An unexpected error occurred.")

Please enter a topic, math problem, or travel query: Interstellar
---PLANNER---
Planner's Decision: research
---RESEARCHER (with SerpApi Web Search)---
Search Results:
["In Earth's future, a global crop blight and second Dust Bowl are slowly rendering the planet uninhabitable. Professor Brand (Michael Caine), a brilliant NASA physicist, is working on plans to save mankind by transporting Earth's population to a new home via a wormhole. But first, Brand must send former NASA pilot Cooper (Matthew McConaughey) and a team of researchers through the wormhole and across the galaxy to find out which of three planets could be mankind's new home.… MORE", 'Interstellar type: PG-13 2014 ‧ Sci-fi/Adventure ‧ 2h 49m.', 'Interstellar entity_type: film, tvm.', 'Interstellar kgmid: /m/0fkf28.', 'Interstellar release_date: October 26, 2014 (USA).', 'Interstellar director: Christopher Nolan.', 'Interstellar running_time: 2h 49m.', 'Interstellar music_composed_by: Hans Zimmer.', 'Interstellar budget: 16